# EU Member States - National Parliament Disclosures

**Countries:** Germany, France, Italy, Spain, Netherlands

## CLI Commands
```bash
mcli run eu-states run                 # Run all countries
mcli run eu-states run --country germany  # Specific country
mcli run eu-states status              # Check status
mcli run eu-states list-countries      # List available countries
```

In [ ]:
import asyncio
import json
import logging
import sys
from datetime import datetime
from functools import wraps
from pathlib import Path
from typing import Optional

import click

project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root / 'src'))

from politician_trading.config import WorkflowConfig
from politician_trading.scrapers.scrapers_eu import (
    run_eu_member_states_collection,
    run_germany_collection,
    run_france_collection,
    run_italy_collection,
    run_spain_collection,
    run_netherlands_collection,
)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
@click.group(name="eu-states")
def eu_states():
    """EU member states national parliament disclosures."""
    pass

def click_async(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        return asyncio.run(f(*args, **kwargs))
    return wrapper

config = WorkflowConfig.default()
scraping_config = config.scraping
OUTPUT_DIR = project_root / 'data' / 'raw' / 'eu_member_states'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

COUNTRY_SCRAPERS = {
    'germany': run_germany_collection,
    'france': run_france_collection,
    'italy': run_italy_collection,
    'spain': run_spain_collection,
    'netherlands': run_netherlands_collection,
}

In [ ]:
@eu_states.command(name="run")
@click.option('--country', default=None, help='Specific country to scrape')
@click.option('--output', default=None, help='Output file path')
@click_async
async def run_ingestion(country: Optional[str], output: Optional[str]):
    """Run EU member states disclosure ingestion."""
    all_disclosures = []
    by_country = {}
    
    if country:
        country = country.lower()
        if country not in COUNTRY_SCRAPERS:
            click.echo(f"Unknown country: {country}", err=True)
            click.echo(f"Available: {', '.join(COUNTRY_SCRAPERS.keys())}")
            return
        click.echo(f"Scraping {country}...")
        disclosures = await COUNTRY_SCRAPERS[country](scraping_config)
        all_disclosures.extend(disclosures)
        by_country[country] = len(disclosures)
    else:
        click.echo("Scraping all EU member states...")
        disclosures = await run_eu_member_states_collection(scraping_config)
        all_disclosures.extend(disclosures)
        for d in disclosures:
            c = d.raw_data.get('country', 'Unknown')
            by_country[c] = by_country.get(c, 0) + 1
    
    click.echo(f"\nTotal: {len(all_disclosures)} disclosures")
    for c, count in by_country.items():
        click.echo(f"  {c}: {count}")
    
    output_file = Path(output) if output else OUTPUT_DIR / f'eu_states_{datetime.now().strftime("%Y%m%d")}.json'
    
    serializable = []
    for d in all_disclosures:
        serializable.append({
            'asset_name': d.asset_name,
            'asset_type': d.asset_type,
            'transaction_type': d.transaction_type.value if d.transaction_type else None,
            'disclosure_date': d.disclosure_date.isoformat() if d.disclosure_date else None,
            'amount_min': float(d.amount_range_min) if d.amount_range_min else None,
            'source_url': d.source_url,
            'raw_data': d.raw_data,
        })
    
    with open(output_file, 'w') as f:
        json.dump({
            'metadata': {
                'source': 'eu_member_states',
                'downloaded_at': datetime.now().isoformat(),
                'total_records': len(all_disclosures),
                'by_country': by_country,
            },
            'disclosures': serializable
        }, f, indent=2, default=str)
    click.echo(f"\nSaved to {output_file}")

In [ ]:
@eu_states.command(name="list-countries")
def list_countries():
    """List available EU member state countries."""
    click.echo("Available countries:")
    for country in COUNTRY_SCRAPERS.keys():
        click.echo(f"  - {country}")

In [ ]:
@eu_states.command(name="status")
def check_status():
    """Check status of EU member states ingestion."""
    files = list(OUTPUT_DIR.glob('eu_states_*.json'))
    if files:
        latest = max(files, key=lambda p: p.stat().st_mtime)
        with open(latest) as f:
            data = json.load(f)
        metadata = data.get('metadata', {})
        click.echo(f"Latest: {latest.name}")
        click.echo(f"Records: {metadata.get('total_records', 'Unknown')}")
        click.echo(f"By country: {metadata.get('by_country', {})}")
    else:
        click.echo("No data found.")